In [1]:
from dotenv import load_dotenv
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
import os


load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
SCRAPIN_API_KEY = os.environ["SCRAPIN_API_KEY"]
SERPAPI_API_KEY = os.environ["SERPAPI_API_KEY"]

### 1. Some basic chains

In [3]:
summary_template = """ 
given the information {information} about a person, I want you to create:
1. A short summary
2. Two interesting facts about them
"""

summary_prompt_template = PromptTemplate(
    input_variables=["information"], template=summary_template
)
llm = ChatOpenAI(
    temperature=0, model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY
)
chain = LLMChain(llm=llm, prompt=summary_prompt_template)

In [4]:
information = """ 
Elon Reeve Musk (/ˈiːlɒn/; EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world, with an estimated net worth of US$190 billion as of March 2024, according to the Bloomberg Billionaires Index, and $195 billion according to Forbes, primarily from his ownership stakes in Tesla and SpaceX.[5][6]

A member of the wealthy South African Musk family, Elon was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the University of Pennsylvania, and received bachelor's degrees in economics and physics. He moved to California in 1995 to attend Stanford University, but dropped out after two days and, with his brother Kimbal, co-founded online city guide software company Zip2. The startup was acquired by Compaq for $307 million in 1999, and that same year Musk co-founded X.com, a direct bank. X.com merged with Confinity in 2000 to form PayPal.

In October 2002, eBay acquired PayPal for $1.5 billion, and that same year, with $100 million of the money he made, Musk founded SpaceX, a spaceflight services company. In 2004, he became an early investor in electric vehicle manufacturer Tesla Motors, Inc. (now Tesla, Inc.). He became its chairman and product architect, assuming the position of CEO in 2008. In 2006, Musk helped create SolarCity, a solar-energy company that was acquired by Tesla in 2016 and became Tesla Energy. In 2013, he proposed a hyperloop high-speed vactrain transportation system. In 2015, he co-founded OpenAI, a nonprofit artificial intelligence research company. The following year, Musk co-founded Neuralink—a neurotechnology company developing brain–computer interfaces—and the Boring Company, a tunnel construction company. In 2022, he acquired Twitter for $44 billion. He subsequently merged the company into newly created X Corp. and rebranded the service as X the following year. In March 2023, he founded xAI, an artificial intelligence company.

Musk has expressed views that have made him a polarizing figure.[7] He has been criticized for making unscientific and misleading statements, including COVID-19 misinformation and antisemitic conspiracy theories.[7][8][9][10] His ownership of Twitter has been similarly controversial, being marked by the laying off of a large number of employees, an increase in hate speech and misinformation and disinformation on the website, as well as changes to Twitter Blue verification. In 2018, the U.S. Securities and Exchange Commission (SEC) sued him, alleging that he had falsely announced that he had secured funding for a private takeover of Tesla. To settle the case, Musk stepped down as the chairman of Tesla and paid a $20 million fine.
"""

In [5]:
res = chain.invoke(input={"information": information})

In [6]:
print(res["text"])

1. Elon Musk is a South African-born businessman and investor known for founding companies such as SpaceX, Tesla, and the Boring Company. He is one of the wealthiest individuals in the world, with a net worth of over $190 billion.

2. Two interesting facts about Elon Musk:
- He dropped out of Stanford University after just two days to pursue entrepreneurial ventures.
- Musk has been involved in various industries, including space exploration, electric vehicles, artificial intelligence, and tunnel construction.


### 2. Demo with our custom LinkedIn scrapping function 

In [9]:
from third_parties.linkedin import scrape_linkedin_profile
from agents.linkedin_lookup_agent import lookup as linkedin_lookup_agent

In [8]:
def ice_break(name: str) -> str:
    linkedin_profile_url = linkedin_lookup_agent(
        name=name, openai_api_key=OPENAI_API_KEY
    )
    summary_template = """ 
    given the information {information} about a person, I want you to create:
    1. A short summary
    2. Two interesting facts about them
    3. A topic that may interest them
    4. 2 creative ice breakers to open a conversation with them
    """
    summary_prompt_template = PromptTemplate(
        input_variables=["information"], template=summary_template
    )
    llm = ChatOpenAI(
        temperature=0, model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY
    )
    chain = LLMChain(llm=llm, prompt=summary_prompt_template)
    linkedin_data = scrape_linkedin_profile(
        linkedin_profile_url=linkedin_profile_url, scrapin_api_key=SCRAPIN_API_KEY
    )
    result = chain.run(information=linkedin_data)
    print(result)
    return result


# note that we dont need to explicitly pass the "SERPAPI_API_KEY" to the function because it is already set as an environment variable
# this can be applied to the OpenAI API key as well, but it is passed explicitly for demonstration purposes

In [21]:
result = ice_break("Joshua Phuong Le")



> Entering new AgentExecutor chain...
I should use the tool to crawl Google for the LinkedIn profile page of Joshua Phuong Le.
Action: Crawl Google for linkedin profile page
Action Input: Joshua Phuong Le LinkedIn profile
Observation: ["I'm interested in data science, machine learning, PC/tech/geeky stuff, and writing about what I learnt. I'm currently learning aspects of cloud-based ML ...", 'Founder of the AiEdge | Follow me to learn about Machine Learning Engineering, Machine Learning System Design, MLOps, and the latest techniques ...', "Joshua Phuong Le's Post. View profile for Joshua Phuong Le. Joshua Phuong Le. Data Scientist at PSA International. 6mo. Report this post; Close ...", 'Joshua Phuong Le. Data Scientist at PSA International | Data Science, Machine Learning and Writing. Singapore. PSA International Pte Ltd, +9 more.', "Hi, I'm Joshua; I'm interested in data science, machine learning, and PC/tech/geeky stuff; I'm currently learning aspects of cloud-based ML and ...",

'1. Short summary:\nJoshua Phuong Le is a Data Scientist at PSA International in Singapore. He has a background in Chemical Engineering and holds a Master\'s degree in IT in Business from Singapore Management University. Joshua is passionate about data science, machine learning, and technology.\n\n2. Two interesting facts about Joshua:\n- Joshua has transitioned from Chemical Engineering to pursue a career in data science and machine learning.\n- He has experience working as a Process Design Engineer and Project Development Manager at ExxonMobil before transitioning to his current role as a Data Scientist.\n\n3. A topic that may interest Joshua:\nGiven his background in Chemical Engineering and current role in data science, Joshua may be interested in exploring the intersection of these fields, such as applications of machine learning in chemical processes or data-driven solutions for industrial operations.\n\n4. 2 creative ice breakers to open a conversation with Joshua:\n- "Hi Joshua

### 3. Controling the output format

In [6]:
# now we will make the output serializable and consumable by the frontend
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
from third_parties.linkedin import scrape_linkedin_profile
from agents.linkedin_lookup_agent import lookup as linkedin_lookup_agent


class PersonIntel(BaseModel):
    summary: str = Field(description="A short summary of the person")
    facts: List[str] = Field(description="Interesting facts about the person")
    topics_of_interest: List[str] = Field(
        description="Topics that may interest the person"
    )
    ice_breakers: List[str] = Field(
        description="Creative ice breakers to open a conversation with the person"
    )

    def to_dict(self):
        return {"summary": self.summary, "facts": self.facts, "topics_of_interest": self.topics_of_interest, "ice_breakers": self.ice_breakers}



def ice_break(name: str, parser:PydanticOutputParser) -> PersonIntel:
    linkedin_profile_url = linkedin_lookup_agent(
        name=name, openai_api_key=OPENAI_API_KEY
    )
    summary_template = """ 
    given the information {information} about a person, I want you to create:
    1. A short summary
    2. Two interesting facts about them
    3. A topic that may interest them
    4. 2 creative ice breakers to open a conversation with them 
    \n{format_instructions}
    """
    summary_prompt_template = PromptTemplate(
        input_variables=["information"], 
        template=summary_template,
        partial_variables={"format_instructions":parser.get_format_instructions()}
    )
    llm = ChatOpenAI(
        temperature=0, model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY
    )
    chain = LLMChain(llm=llm, prompt=summary_prompt_template)
    linkedin_data = scrape_linkedin_profile(
        linkedin_profile_url=linkedin_profile_url, scrapin_api_key=SCRAPIN_API_KEY
    )
    result = chain.run(information=linkedin_data)
    print(result)
    return parser.parse(result)

In [10]:
person_intel_parser = PydanticOutputParser(pydantic_object=PersonIntel)
person = ice_break("Joshua Phuong Le", person_intel_parser)



> Entering new AgentExecutor chain...
I should use the tool to crawl Google for the LinkedIn profile page of Joshua Phuong Le.
Action: Crawl Google for linkedin profile page
Action Input: Joshua Phuong Le LinkedIn profile
Observation: ["Hi, I'm Joshua I'm interested in data science, machine learning, PC/tech/geeky stuff, and writing about what I learnt. I'm currently learning aspects ...", 'View profile for Renuka P. Renuka P. Senior Data Scientist with hands-on experience|| 4K followers || 3.2 M+ impressions|| || content creator on ...', 'Resuming to my blogging hobby, this article is my personal experience when transitioning from engineering to data science.', "Joshua Phuong Le's Post. View profile for Joshua Phuong Le. Joshua Phuong Le. Data Scientist at PSA International. 7mo. Report this post; Close ...", "Recently I've been working on a very commonly seen text clustering problem, which requires the grouping of texts based on their content and ...", "Hi, I'm Joshua; I'm interest

In [11]:
person.to_dict()

{'summary': 'Joshua Phuong Le is a Data Scientist at PSA International in Singapore with a background in Chemical Engineering. He is passionate about data science, machine learning, and technology.',
 'facts': ['Joshua has transitioned from Chemical Engineering to pursue a career in data science and machine learning.',
  "He has a Master's degree in IT in Business from Singapore Management University."],
 'topics_of_interest': ['Data Science',
  'Machine Learning',
  'Cloud-based ML',
  'Software Engineering'],
 'ice_breakers': ['Hi Joshua! I noticed your transition from Chemical Engineering to Data Science, what inspired you to make that change?',
  'Hey Joshua, I see you have a background in IT in Business, how has that influenced your approach to data science projects?']}